In [1]:
%%file myConv.py

import torch
import numpy as np
def myConv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #groups exeption
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad3d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad3d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad3d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad3d(padding)
      matrix = pad(matrix)

    #генерация ядра
    if type(kernel_size) == tuple:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1], kernel_size[2])
    if type(kernel_size) == int:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size, kernel_size)

    #filter = b
    # bias_val = bi

    llist = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for k in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride):
        for i in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
          for j in range (0, matrix.shape[3]-((filter.shape[4]-1)*dilation+1)+1, stride):
            summa = 0
            for c in range (in_channels//groups): #groups
              if groups>1:
                val = matrix[l*(in_channels//groups)+c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
              else:
                val = matrix[c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
              mini_sum = (val*filter[l][c]).sum()
              summa = summa + mini_sum
            feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

      llist.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1, (matrix.shape[3]-((filter.shape[4]-1)*dilation+1))//stride+1))

    return np.array(llist), torch.tensor(np.array(filter)), torch.tensor(np.array(bias_val))

  return Svertca

Overwriting myConv.py


In [2]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Overwriting conftest.py


In [3]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor1 = torch.rand(4, 6, 6, 6)
tensor2 = torch.rand(3, 24, 28, 20)
tensor3 = torch.rand(1, 1, 1, 1)


def test_1(session):
  myFunction = myConv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  torchFunction.weight.data = kernel
  torchFunction.bias.data = bias_val
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myConv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  torchFunction2.weight.data = kernel2
  torchFunction2.bias.data = bias_val2
  myResult = str(np.round(result2, 2))
  torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult == myResult


def test_3(session):
  myFunction3 = myConv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.Conv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  torchFunction3.weight.data = kernel3
  torchFunction3.bias.data = bias_val3
  myResult = str(np.round(result3, 2))
  torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult == myResult



Overwriting test_http.py


In [4]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0
rootdir: C:\Users\arina\smz
plugins: anyio-3.5.0
collected 3 items

test_http.py ...                                                         [100%]

============================== 3 passed in 7.60s ==============================


In [6]:
import os, sys
import torch
import numpy as np

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor1 = torch.rand(4, 6, 6, 6)

myFunction = myConv3d(in_channels=4, out_channels=2, kernel_size=5, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction(tensor1)
torchFunction = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=5, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val

print(result)
print('===================================')
print(np.array(torchFunction(tensor1).data))

[[[[52.72069168 55.3234787 ]
   [54.61018372 57.5618515 ]]

  [[55.02480316 53.92865753]
   [57.3835144  57.74442291]]]


 [[[68.68241882 68.69812012]
   [72.49376678 69.58637238]]

  [[67.27288818 66.48874664]
   [69.54272461 67.63152313]]]]
[[[[52.720707 55.323494]
   [54.610172 57.56184 ]]

  [[55.02482  53.928654]
   [57.38351  57.744404]]]


 [[[68.6824   68.69811 ]
   [72.49377  69.58638 ]]

  [[67.27287  66.48874 ]
   [69.54273  67.63153 ]]]]


In [7]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor2 = torch.rand(3, 10, 11, 12)

myFunction2 = myConv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=3, groups=1, bias=True, padding_mode='zeros')
result2, kernel2, bias_val2 = myFunction2(tensor2)
torchFunction2 = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=3, groups=1, bias=True, padding_mode='zeros')
torchFunction2.weight.data = kernel2
torchFunction2.bias.data = bias_val2

print(result2)
print('===================================')
print(np.array(torchFunction2(tensor2).data))

[[[[21.19206429 20.96116829 21.67557716 19.97020721 21.68794823
    21.491045  ]
   [20.8784523  21.14939499 20.0027256  19.67279625 19.60270119
    19.87233925]
   [21.72640991 20.41779137 19.98821068 18.83367348 19.03825378
    18.81828499]
   [19.60969734 22.45463371 20.21418762 19.45671272 24.25629997
    21.73147964]
   [19.55188942 21.12685204 18.52770996 21.60019875 19.05060005
    21.0271244 ]]

  [[20.25897408 17.38823128 23.63492203 19.73931885 19.70783043
    21.11930466]
   [22.65858459 20.94630623 21.58201027 22.41709709 20.8094883
    19.98189926]
   [20.1873188  18.33280563 18.43042564 17.3616333  17.30643654
    20.57359886]
   [19.03752136 17.93053627 23.14079094 20.74354935 19.57504082
    21.04667664]
   [21.47668648 20.46988487 20.73752975 23.50160599 21.05413437
    18.10944939]]

  [[21.80059052 24.17033958 20.56357002 20.95233917 22.75702286
    20.1516819 ]
   [21.46239471 20.67054939 21.59744453 22.25643349 20.27157402
    21.83917236]
   [20.54981041 19.302448

In [8]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor3 = torch.rand(1, 1, 1, 1)

myFunction3 = myConv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
result3, kernel3, bias_val3 = myFunction3(tensor3)
torchFunction3 = torch.nn.Conv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = kernel3
torchFunction3.bias.data = bias_val3

print(result3)
print('===================================')
print(np.array(torchFunction3(tensor3).data))

[[[[0.20939991]]]]
[[[[0.20939991]]]]


In [9]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor4 = torch.rand(4, 5, 8, 5)

myFunction4 = myConv3d(in_channels=4, out_channels=2, kernel_size=3, stride=3, padding=1, dilation=1, groups=2, bias=True, padding_mode='zeros')
result4, kernel4, bias_val4 = myFunction4(tensor4)
torchFunction4 = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=3, stride=3, padding=1, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction4.weight.data = kernel4
torchFunction4.bias.data = bias_val4

print(result4)
print('===================================')
print(np.array(torchFunction4(tensor4).data))

[[[[ 3.24746704  4.17873526]
   [ 6.36931181  9.53795338]
   [ 5.61576605  7.87391281]]

  [[ 5.19947815  9.81392956]
   [ 9.97983456 16.55055809]
   [ 9.16078663 13.46387577]]]


 [[[ 5.67966461  7.78256416]
   [ 7.98002195  9.16406918]
   [ 7.838346    9.29517841]]

  [[ 7.27510881  9.51250744]
   [ 9.04335785 13.40098286]
   [10.1187849  13.60484982]]]]
[[[[ 3.2474666  4.178735 ]
   [ 6.369312   9.537954 ]
   [ 5.6157656  7.8739133]]

  [[ 5.1994777  9.81393  ]
   [ 9.979834  16.550556 ]
   [ 9.160786  13.463878 ]]]


 [[[ 5.6796646  7.782564 ]
   [ 7.9800224  9.164068 ]
   [ 7.838347   9.295178 ]]

  [[ 7.275109   9.512507 ]
   [ 9.043357  13.400985 ]
   [10.118784  13.60485  ]]]]
